# Prophet - Predecir el valor de Bitcoin


Prophet es una de las librerías más avanzadas para predecir series temporales desarrollada por Facebook. Te enseñaremos a como entrenar un modelo con Prophet, a optimizarlo y a utilizarlo para realizar predicciones futuras. En este ejercicio vamos a practicar a predecir el valor de Bitcoin, una criptomoneda. Es la criptomoneda que le ha marcado el camino a todas las demás que llegaron después utilizando su tecnología.  

<hr/>
<div class="alert alert-success alertsuccess" style="margin-top: 20px">
[Tip]: Para ejecutar el código de Python en la celda de código a continuación, haz clic en la celda para seleccionarla y presiona <kbd>Shift</kbd> + <kbd>Enter</kbd>.
</div>
<hr/>


In [ ]:
#Dependencias de Prophet
!pip install pystan==2.19.1.1 --quiet
!pip install cmdstanpy --quiet
# Instalamos el modelo predictivo Prophet
!pip install fbprophet --quiet
# De aquí vamos a descargar los datos
!pip install yfinance --quiet

In [ ]:
#Importamos las dependencias
import pandas as pd
import yfinance as yf
from datetime import datetime
from datetime import timedelta
import plotly.graph_objects as go
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, plot_components_plotly
import warnings

warnings.filterwarnings('ignore')

pd.options.display.float_format = '${:,.2f}'.format

<h3 id="version">Cargar histórico de datos</h3>


<p>
    Para descargarnos la serie temporal de BTC lo vamos a hacer desde Yahoo Finance. En concreto vamos a descargar el histórico desde 2016 hasta la actualidad.
</p>



In [ ]:
# Fecha de inicio es un string 2016-01-01 y fecha de fin (hoy) en formato texto '%Y-%m-%d'
today = '2023-01-11'
start_date = '2016-01-01'
# Descargar el dataframe
gold_df = yf.download('GC=F',start_date, today).reset_index()

gold_df.tail()

[*********************100%***********************]  1 of 1 completed


,Date,Open,High,Low,Close,Adj Close,Volume
1761,2023-01-04,"$1,845.60","$1,859.10","$1,845.60","$1,852.80","$1,852.80",25
1762,2023-01-05,"$1,855.20","$1,855.20","$1,834.80","$1,834.80","$1,834.80",24
1763,2023-01-06,"$1,838.40","$1,868.20","$1,835.30","$1,864.20","$1,864.20",26
1764,2023-01-09,"$1,867.00","$1,880.00","$1,867.00","$1,872.70","$1,872.70",62
1765,2023-01-10,"$1,877.80","$1,878.10","$1,871.60","$1,871.60","$1,871.60",62


In [ ]:
gold_df

In [ ]:
# Validamos que no hay datos vacíos
gold_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1766 entries, 0 to 1765
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       1766 non-null   datetime64[ns]
 1   Open       1766 non-null   float64       
 2   High       1766 non-null   float64       
 3   Low        1766 non-null   float64       
 4   Close      1766 non-null   float64       
 5   Adj Close  1766 non-null   float64       
 6   Volume     1766 non-null   int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 96.7 KB


In [ ]:
gold_df.isnull().sum()   

Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64

In [ ]:
df = gold_df
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-01-04,"$1,063.40","$1,082.50","$1,063.20","$1,075.10","$1,075.10",143
1,2016-01-05,"$1,075.60","$1,081.50","$1,075.30","$1,078.40","$1,078.40",82
2,2016-01-06,"$1,081.60","$1,093.70","$1,081.60","$1,091.90","$1,091.90",52
3,2016-01-07,"$1,091.60","$1,109.40","$1,091.60","$1,107.70","$1,107.70",122
4,2016-01-08,"$1,111.10","$1,111.10","$1,093.00","$1,097.80","$1,097.80",98
...,...,...,...,...,...,...,...
1761,2023-01-04,"$1,845.60","$1,859.10","$1,845.60","$1,852.80","$1,852.80",25
1762,2023-01-05,"$1,855.20","$1,855.20","$1,834.80","$1,834.80","$1,834.80",24
1763,2023-01-06,"$1,838.40","$1,868.20","$1,835.30","$1,864.20","$1,864.20",26
1764,2023-01-09,"$1,867.00","$1,880.00","$1,867.00","$1,872.70","$1,872.70",62


In [ ]:
df = gold_df[['Date','Open']]
df

,Date,Open
0,2016-01-04,"$1,063.40"
1,2016-01-05,"$1,075.60"
2,2016-01-06,"$1,081.60"
3,2016-01-07,"$1,091.60"
4,2016-01-08,"$1,111.10"
...,...,...
1761,2023-01-04,"$1,845.60"
1762,2023-01-05,"$1,855.20"
1763,2023-01-06,"$1,838.40"
1764,2023-01-09,"$1,867.00"


In [ ]:
# TAREA: Filtramos el dataframe para quedarnos solo las columnas "Date" y "Open"
# Date será la fecha del valor y Open el valor del oro en ese momento

# Valores que espera el prophet
# Eje X: Indíce llamado ds
# Eje Y: Serie a predecir llamada y
# TAREA HECHA: Crear un diccionario con clave "Date" y valor "ds"
# Y otra clave "Open" y valor "y"
datos = {
    "Date": "ds", 
    "Open": "y",
}

# TAREA: Utiliza el diccionario para renombrar las columnas de df

df = df.rename(columns=datos)
df

,ds,y
0,2016-01-04,"$1,063.40"
1,2016-01-05,"$1,075.60"
2,2016-01-06,"$1,081.60"
3,2016-01-07,"$1,091.60"
4,2016-01-08,"$1,111.10"
...,...,...
1761,2023-01-04,"$1,845.60"
1762,2023-01-05,"$1,855.20"
1763,2023-01-06,"$1,838.40"
1764,2023-01-09,"$1,867.00"


In [ ]:
#Ejecuta la siguiente línea, si lo has hecho bien hasta aquí no dará error
df['ds'] = df['ds'].dt.tz_localize(None)

In [ ]:
df.tail()

,ds,y
1761,2023-01-04,"$1,845.60"
1762,2023-01-05,"$1,855.20"
1763,2023-01-06,"$1,838.40"
1764,2023-01-09,"$1,867.00"
1765,2023-01-10,"$1,877.80"


In [ ]:
# Crear la gráfica del precio de apertura
#Asigna a la variable x la columna ds
x = df["ds"]
#Asigna a la variable y la columna y
y = df["y"]

fig = go.Figure()

fig.add_trace(go.Scatter(x=x, y=y))

# Le ponemos el título a la gráfica
titulo = "Prediccion ORO"
fig.update_layout(
    title_text=titulo,
)

fig.update_layout(
    xaxis=dict(
        rangeselector=dict(
            buttons=list(
                [
                    dict(count=1, label="1m", step="month", stepmode="backward"),
                    dict(count=6, label="6m", step="month", stepmode="backward"),
                    dict(count=1, label="YTD", step="year", stepmode="todate"),
                    dict(count=1, label="1a", step="year", stepmode="backward"),
                    dict(step="all"),
                ]
            )
        ),
        rangeslider=dict(visible=True),
        type="date",
    )
)

<h3 id="version">Entrenar y predecir el modelo</h3>


<p>
    Ahora que has visto en la gráfica de que datos partimos. Inicializamos el Prophet y lo entramos pasandole el dataframe
</p>



In [ ]:
#TAREA: Inicializa Prophet en la variable m con seasonality_mode en modo "multiplicative"
m = Prophet()


In [ ]:
#TAREA: Entrena el modelo m con el dataframe de datos df
m.fit(df)

INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


<p>
    Le decimos al modelo m los días en futuro que queremos predecir, en este caso el próximo año.
</p>



In [ ]:
# TAREA: generar un dataframe con los días que tenemos y los 365 días siguientes a hoy para predecirlo
# PISTA: puedes usar la función del prophet make_future_dataframe()
future = m.make_future_dataframe(periods=365)
future.tail()

,ds
2126,2024-01-06
2127,2024-01-07
2128,2024-01-08
2129,2024-01-09
2130,2024-01-10


In [ ]:
# En el caso que hayas elegido un valor o fondo tradicional, es decir, que no opere los fines
# de semana, descomenta la siguiente línea:

future = future[ future['ds'].dt.dayofweek < 5 ] # Nos elimina los fines de semana de 'ds' para no predecirlos
future

,ds
0,2016-01-04
1,2016-01-05
2,2016-01-06
3,2016-01-07
4,2016-01-08
...,...
2124,2024-01-04
2125,2024-01-05
2128,2024-01-08
2129,2024-01-09


<p>
    Al modelo entrenado le pasamos el dataframe a predecir.
</p>



In [ ]:
#Utiliza el modelo m para predecir el dataframe future
forecast = m.predict(future)

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

,ds,yhat,yhat_lower,yhat_upper
2022,2024-01-04,"$1,734.80","$1,269.46","$2,179.50"
2023,2024-01-05,"$1,735.71","$1,277.79","$2,198.56"
2024,2024-01-08,"$1,738.19","$1,284.19","$2,199.16"
2025,2024-01-09,"$1,737.38","$1,254.05","$2,193.42"
2026,2024-01-10,"$1,737.53","$1,273.07","$2,186.48"


<p>
    Gráfica con el modelo entrenado y los valores de un año en futuro predecidos.
</p>



In [ ]:
next_day = (datetime.today() + timedelta(days=1)).strftime('%Y-%m-%d')
forecast[forecast['ds'] == next_day]['yhat'].item()
plot_plotly(m, forecast)

In [ ]:
plot_components_plotly(m, forecast)